In [1441]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1442]:
import numpy as np
import pandas as pd
import nltk
import unicodedata
import re
import numpy as np
from numpy import dot
from numpy.linalg import norm
from gensim.models import Word2Vec
from nltk import word_tokenize
from scipy.sparse import hstack
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [1443]:
df_train = pd.read_csv("drive/My Drive/UST/6000H/ml4nlp-argimpact/train.csv")
df_test = pd.read_csv("drive/My Drive/UST/6000H/ml4nlp-argimpact/test.csv")
df_valid = pd.read_csv("drive/My Drive/UST/6000H/ml4nlp-argimpact/valid.csv")

In [1444]:
# df_train = df_train.append(df_valid)

In [1445]:
class_names = df_train.impact_label.unique().tolist()

In [1446]:
df_train['context'] = df_train.apply(lambda row: ' '.join(eval(row.context)), axis=1)
df_test['context'] = df_test.apply(lambda row: ' '.join(eval(row.context)), axis=1)
df_valid['context'] = df_valid.apply(lambda row: ' '.join(eval(row.context)), axis=1)

df_test['text'] = df_test.apply(lambda row: eval(row.text)[0], axis=1)

In [1447]:
df_train['impact_label'] = df_train.impact_label.map({"NOT_IMPACTFUL": 0, "MEDIUM_IMPACT": 1, "IMPACTFUL": 2})
df_valid['impact_label'] = df_valid.impact_label.map({"NOT_IMPACTFUL": 0, "MEDIUM_IMPACT": 1, "IMPACTFUL": 2})

In [1448]:
df_train['cleaned_stance_label'] = df_train['stance_label'].apply(lambda x: ' '.join(eval(x)[1:]))
df_test['cleaned_stance_label'] = df_test['stance_label'].apply(lambda x: ' '.join(eval(x)[1:]))
df_valid['cleaned_stance_label'] = df_valid['stance_label'].apply(lambda x: ' '.join(eval(x)[1:]))

In [1449]:
df_train["text"] = df_train["context"] + " " + df_train["text"]
df_test["text"] = df_test["context"] + " " + df_test["text"]
df_valid["text"] = df_valid["context"] + " " + df_valid["text"]

In [1450]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn import metrics

In [1451]:
text_count_vect = CountVectorizer(lowercase=True, stop_words="english", ngram_range=(1, 4), min_df=0.005)
# context_count_vect = CountVectorizer(lowercase=True, stop_words="english", ngram_range=(1, 5))
stance_count_vect = CountVectorizer(lowercase=True, ngram_range=(1, 5))

In [1452]:
X_train = text_count_vect.fit_transform(df_train['text'].to_numpy())
X_valid = text_count_vect.transform(df_valid['text'].to_numpy())
# X_test = text_count_vect.transform(df_test['text'].to_numpy())

In [1453]:
# train_context_count_vect = context_count_vect.fit_transform(df_train['context'].to_numpy())
# valid_context_count_vect = context_count_vect.transform(df_valid['context'].to_numpy())
# test_context_count_vect = context_count_vect.transform(df_test['context'].to_numpy())

In [1454]:
train_stance = stance_count_vect.fit_transform(df_train['cleaned_stance_label'].to_numpy())
valid_stance = stance_count_vect.transform(df_valid['cleaned_stance_label'].to_numpy())
# test_stance = stance_count_vect.transform(df_test['cleaned_stance_label'].to_numpy())

In [1455]:
tfidf_text = TfidfVectorizer(max_features = 10000, ngram_range=(1,4))
# tfidf_context = TfidfVectorizer(max_features = 10000, ngram_range=(1,4))

In [1456]:
train_text_tfidf_vect = tfidf_text.fit_transform(df_train['text'].to_numpy())
valid_text_tfidf_vect = tfidf_text.transform(df_valid['text'].to_numpy())

In [1457]:
# train_context_tfidf_vect = tfidf_context.fit_transform(df_train['context'].to_numpy())
# valid_context_tfidf_vect = tfidf_context.transform(df_valid['context'].to_numpy())

In [1458]:
# X_train = hstack((X_train, train_context_count_vect))
# X_valid = hstack((X_valid, valid_context_count_vect))
# # X_test = hstack((X_test, test_context_count_vect))

X_train = hstack((X_train, train_stance))
X_valid = hstack((X_valid, valid_stance))
# X_test = hstack((X_test, test_stance))

# X_train = hstack((X_train, train_text_tfidf_vect))
# X_valid = hstack((X_valid, valid_text_tfidf_vect))

# X_train = hstack((X_train, train_context_tfidf_vect))
# X_valid = hstack((X_valid, valid_context_tfidf_vect))

In [1459]:
y_train = df_train.impact_label.to_numpy()
y_valid = df_valid.impact_label.to_numpy()

In [1460]:
sgd = SGDClassifier(random_state=42)
rfc = RandomForestClassifier(random_state=42)

In [1461]:
sgd.fit(X_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=42, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [1462]:
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [1463]:
validation_pred = sgd.predict(X_valid)
print(f"SGD F-1 score: {metrics.f1_score(df_valid.impact_label.to_numpy(), validation_pred, average='macro')}")

validation_pred = rfc.predict(X_valid)
print(f"RF F-1 score: {metrics.f1_score(df_valid.impact_label.to_numpy(), validation_pred, average='macro')}")

SGD F-1 score: 0.5365381634422253
RF F-1 score: 0.6014260746366586


In [1401]:
# text_clf.fit(df_train.text.to_numpy(), df_train.impact_label.to_numpy())

In [ ]:
# validation_pred = text_clf.predict(df_valid.text.to_numpy())

In [ ]:
# print(f"F-1 score: {metrics.f1_score(df_valid.impact_label.to_numpy(), validation_pred, average='macro')}")

In [ ]:
# metrics.classification_report(df_valid.impact_label.to_numpy(), validation_pred, target_names=class_names)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
parameters = {
    'feats__vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'clf__alpha': (0.02, 0.03, 0.2),
}
gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1, verbose=1)

In [ ]:
gs_clf.fit(df_train.text.to_numpy(), df_train.impact_label.to_numpy())

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   19.7s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('feats',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('vect',
                                                                        CountVectorizer(analyzer='word',
                                                                                        binary=False,
                                                                                        decode_error='strict',
                                                                                        dtype=<class 'numpy.int64'>,
                                                                                        encoding='utf-8',
                                                                                        input='content',
                                                                              

In [ ]:
gs_clf.cv_results_

{'mean_fit_time': array([0.33352962, 1.07944336, 2.03904166, 0.29629989, 1.03692803,
        1.89910464]),
 'mean_score_time': array([0.07668324, 0.15674677, 0.21972637, 0.06506205, 0.14441047,
        0.1965364 ]),
 'mean_test_score': array([0.61257253, 0.61121857, 0.60754352, 0.54158607, 0.5655706 ,
        0.59748549]),
 'param_clf__alpha': masked_array(data=[0.01, 0.01, 0.01, 0.001, 0.001, 0.001],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_feats__vect__ngram_range': masked_array(data=[(1, 1), (1, 2), (1, 3), (1, 1), (1, 2), (1, 3)],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'clf__alpha': 0.01, 'feats__vect__ngram_range': (1, 1)},
  {'clf__alpha': 0.01, 'feats__vect__ngram_range': (1, 2)},
  {'clf__alpha': 0.01, 'feats__vect__ngram_range': (1, 3)},
  {'clf__alpha': 0.001, 'feats__vect__ngram_range': (1, 1)},
  {'clf__al

In [ ]:
validation_pred = gs_clf.predict(df_valid.text.to_numpy())

In [ ]:
print(f"F-1 score: {metrics.f1_score(df_valid.impact_label.to_numpy(), validation_pred, average='macro')}")

F-1 score: 0.4852736607231976


In [ ]:
gs_clf.fit(df_train.text.to_numpy(), df_train.impact_label.to_numpy())

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   24.3s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('feats',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('vect',
                                                                        CountVectorizer(analyzer='word',
                                                                                        binary=False,
                                                                                        decode_error='strict',
                                                                                        dtype=<class 'numpy.int64'>,
                                                                                        encoding='utf-8',
                                                                                        input='content',
                                                                              

In [ ]:
gs_clf.cv_results_

{'mean_fit_time': array([0.40860076, 1.26529689, 2.40461206, 0.36888561, 1.25734696,
        2.24724402]),
 'mean_score_time': array([0.09286661, 0.17571011, 0.27506499, 0.08236485, 0.17906299,
        0.25027771]),
 'mean_test_score': array([0.6153257 , 0.62280889, 0.62185424, 0.55033433, 0.60178851,
        0.60895539]),
 'param_clf__alpha': masked_array(data=[0.01, 0.01, 0.01, 0.001, 0.001, 0.001],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_feats__vect__ngram_range': masked_array(data=[(1, 1), (1, 2), (1, 3), (1, 1), (1, 2), (1, 3)],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'clf__alpha': 0.01, 'feats__vect__ngram_range': (1, 1)},
  {'clf__alpha': 0.01, 'feats__vect__ngram_range': (1, 2)},
  {'clf__alpha': 0.01, 'feats__vect__ngram_range': (1, 3)},
  {'clf__alpha': 0.001, 'feats__vect__ngram_range': (1, 1)},
  {'clf__al

In [299]:
predictions = sgd.predict(X_test)

In [300]:
submission = df_test[['id']]

In [301]:
submission['pred'] = predictions

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [302]:
submission.to_csv('drive/My Drive/UST/6000H/sgd.csv', index=False)